In [1]:
import asyncio
import pandas as pd
from datetime import datetime, time, timedelta
from zoneinfo import ZoneInfo
from alpaca.data.live.stock import StockDataStream
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetCalendarRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.enums import Adjustment

from IPython.display import clear_output


# Importing the modules themselves for reloading
import TradePosition
import TouchArea
import LiveTrader

# Reloading the modules to apply any changes
import importlib
importlib.reload(TradePosition)
importlib.reload(TouchArea)
importlib.reload(LiveTrader)


from TradePosition import TradePosition, SubPosition, export_trades_to_csv
from TouchArea import TouchArea, TouchAreaCollection
from LiveTrader import LiveTrader

import os, toml
from dotenv import load_dotenv

STANDARD_DATETIME_STR = '%Y-%m-%d %H:%M:%S'
ROUNDING_DECIMAL_PLACES = 10  # Choose an appropriate number of decimal places

load_dotenv(override=True)
livepaper = os.getenv('LIVEPAPER')
config = toml.load('../config.toml')

# Replace with your Alpaca API credentials
API_KEY = config[livepaper]['key']
API_SECRET = config[livepaper]['secret']


debug = True

def debug_print(*args, **kwargs):
    if debug:
        print(*args, **kwargs)

In [2]:
async def main():
    symbol = "AAPL"
    initial_balance = 10000
    trader = LiveTrader(API_KEY, API_SECRET, symbol, initial_balance)
    
    try:
        await asyncio.wait_for(trader.run(), timeout=30)  # 30 seconds timeout
    except asyncio.TimeoutError:
        print("Operation timed out. This is expected outside of market hours.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        await trader.data_stream.close()
        print("Trader stopped.")


In [3]:
import tracemalloc
tracemalloc.start()
import logging
logging.getLogger('alpaca').setLevel(logging.DEBUG)

# Run the main function
clear_output(wait=True)  # Clear previous output

# Check if we're already in an event loop
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    # If we're not in an event loop, create one and run our main function
    asyncio.run(main())
else:
    # If we're already in an event loop, create a task
    loop.create_task(main())

self.initialize_data()...
initialize_data - DEBUG MODE DATES: 2024-07-24 09:30:00 2024-07-24 16:00:00
Fetching historical data from 2024-07-24 09:30:00 to 2024-07-24 16:00:00
DatetimeIndex(['2024-07-24 09:32:00', '2024-07-24 09:33:00',
               '2024-07-24 09:37:00', '2024-07-24 09:43:00',
               '2024-07-24 09:49:00', '2024-07-24 09:50:00',
               '2024-07-24 09:57:00', '2024-07-24 09:58:00',
               '2024-07-24 10:01:00', '2024-07-24 10:06:00',
               ...
               '2024-07-24 15:51:00', '2024-07-24 15:52:00',
               '2024-07-24 15:53:00', '2024-07-24 15:54:00',
               '2024-07-24 15:55:00', '2024-07-24 15:56:00',
               '2024-07-24 15:57:00', '2024-07-24 15:58:00',
               '2024-07-24 15:59:00', '2024-07-24 16:00:00'],
              dtype='datetime64[ns]', name='timestamp', length=312, freq=None)
2024-07-24 16:00:00 2024-07-24 16:00:00 0.0
Initialized historical data: 312 bars
Data range: 2024-07-24 09:32:00 to